In [ ]:
import numpy as np
import pandas as pd

# *Any time you see `____` and `#TODO: Fill in here`, it means you should fill in the blank.*

## 1. Getting the data.

We'll be using the file 'processed.cleveland.data', which is from https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/. It should already be in the .zip file you downloaded.

For more information abouot the dataset, see the description here: https://archive.ics.uci.edu/ml/datasets/heart+Disease and here: https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/heart-disease.names

Run these cells to save the data into a numpy array.

In [ ]:
columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]

In [ ]:
data = np.genfromtxt("processed.cleveland.data", delimiter=",")

In [ ]:
data = np.delete(data, np.where(np.isnan(data).any(axis=1))[0], axis=0)

In [ ]:
data_df = pd.DataFrame(data, columns=columns)

Take a look at the dataset:

In [ ]:
data_df.head

In [ ]:
print(data.shape)

In [ ]:
data.shape

**How many examples are in this dataset?**

**What are the possible labels?**

In [ ]:
labels = [int(label) for label in data[:,-1]]

In [ ]:
np.unique(labels)

**How many of each label are there?**

In [ ]:
for label in np.unique(labels):
    print("number of label %d: %d" % (label, np.count_nonzero(labels == label)))

Read the Data Set Information section here: https://archive.ics.uci.edu/ml/datasets/heart+Disease. **What are we trying to classify?**

Now, we'll make a simplified dataset based on our desired classification.

In [ ]:
labels = data[:,-1]
labels[labels > 0] = 1
data[:,-1] = labels

**Now what are the possible labels? How many of each label do we have? What do each of these labels mean?**

In [ ]:
labels = ____ # TODO: Fill in here.
np.unique(____) # TODO: Fill in here.

In [ ]:
for label in np.unique(labels):
    print("number of label %d: %d" % (label, np.count_nonzero(labels == label)))

**What is the chance rate if we randomly guess a label for each input? What if we always guess the most common label?**

In [ ]:
____/____  # TODO: Fill in here.

**What accuracy do you think we'll be able to get using K-NN or a decision tree?**

## 2. Splitting into train and test sets.

Run the following cell to split our data into a train and test set. We'll start by using 80% of our data for the train set and 20% for the test set. If you have time, feel free to experiment with different train-test splits.

In [ ]:
num_examples = data.shape[0]
indices = np.random.permutation(num_examples)

train_indices = indices[:int(num_examples * 4 / 5)]
test_indices = indices[int(num_examples * 4 / 5):]

train_data = data[____] # TODO: Fill in here.
test_data = data[____] # TODO: Fill in here.

**How many train and test examples do we have?**

In [ ]:
print(train_data.____) # TODO: Fill in here.

In [ ]:
print(test_data.____) # TODO: Fill in here.

**Which columns contain each example's features, and which column contains the label? What are the attributes? (hint: see https://archive.ics.uci.edu/ml/datasets/heart+Disease for more information about the dataset.)**

## 3. Implementing k-Nearest Neighbors.

### 3a. Nearest Neighbors.

First, we'll need to write some functions to compute distance. The following function computes the Euclidean distance between two points.

In [ ]:
def euclidean_distance(example1, example2):
    squared_diff_sum = 0
    for example1_feature, example2_feature in zip(example1, example2):
        squared_diff_sum += (example1_feature - example2_feature) ** 2
    euclidean_distance = np.sqrt(squared_diff_sum)
    return euclidean_distance

Next, we'll write a function to do nearest neighbors classification.

In [ ]:
def nearest_neighbor_classifier(train_inputs, train_labels, test_input):
    closest_index = -1
    smallest_distance = np.inf
    train_size = len(train_inputs)
    for train_index in range(train_size):
        train_input = train_inputs[train_index]
        distance = euclidean_distance(____, ____) # TODO: Fill in here.
        if distance < smallest_distance:
            closest_index = train_index
            smallest_distance = distance
    predicted_label = train_labels[closest_index]
    return predicted_label

Now fill in the following function, which will let us test our predictor's accuracy.

In [ ]:
def test_predictor_accuracy(train_data, test_data):
    train_inputs = train_data[:,:-1]
    train_labels = train_data[:,-1]
    test_inputs = test_data[:,:-1]
    test_labels = test_data[:,-1]
    test_size = test_inputs.shape[0]
    correct_predictions = 0
    for test_index in range(test_size):
        test_input = test_inputs[test_index]
        test_label = test_labels[test_index]
        prediction = nearest_neighbor_classifier(____, ____, ____) # TODO: Fill in here.
        if prediction == test_label:
            correct_predictions += ____ # TODO: Fill in here.
    accuracy = ____ / ____ # TODO: Fill in here.
    return accuracy

In [ ]:
test_predictor_accuracy(____, ____) # TODO: Fill in here.

**Is this accuracy higher or lower than what you would have expected?**

### 3b. k-Nearest Neighbors.

Now let's try extending our function to implement k-nearest neighbors. First, we'll need a function that finds the most common label in a list of labels.

In [ ]:
def majority_vote(labels):
    unique_labels = np.unique(labels)
    num_unique_labels = len(unique_labels)
    unique_labels_counts = {label:0 for label in unique_labels}
    for label in labels:
        unique_labels_counts[label] += 1
    most_common_label = list(unique_labels_counts.keys())[np.argmax(list(unique_labels_counts.values()))]
    return most_common_label

Now fill in the following function to implement k-nearest neighbors.

In [ ]:
def k_nearest_neighbor_classifier(train_inputs, train_labels, test_input, k):
    train_size = len(____) # TODO: Fill in here.
    distances = np.zeros(____) # TODO: Fill in here.
    for train_index in range(train_size):
        train_input = train_inputs[train_index]
        distance = euclidean_distance(____, ____) # TODO: Fill in here.
        distances[train_index] = distance
    distance_order = np.argpartition(distances, k)
    k_closest_train_indices = distance_order[:k]
    k_closest_train_labels = train_labels[k_closest_train_indices]
    predicted_label = majority_vote(____) # TODO: Fill in here.
    return predicted_label

Finally, fill in the following function to test our predictor's accuracy.

In [ ]:
def test_knn_predictor_accuracy(train_data, test_data, k):
    train_inputs = train_data[:,:-1]
    train_labels = train_data[:,-1]
    test_inputs = test_data[:,:-1]
    test_labels = test_data[:,-1]
    test_size = test_inputs.shape[0]
    correct_predictions = 0
    for test_index in range(test_size):
        test_input = test_inputs[test_index]
        test_label = test_labels[test_index]
        prediction = k_nearest_neighbor_classifier(____, ____, ____, ____) # TODO: Fill in here.
        if prediction == test_label:
            correct_predictions += ____ # TODO: Fill in here.
    accuracy = ____ / ____ # TODO: Fill in here.
    return accuracy

What is our predictor's accuracy?

In [ ]:
test_knn_predictor_accuracy(train_data, test_data, 1)

### 3c. Nearest neighbors using sklearn.

We just implemented nearest neighbors by ourself, but we can also borrow code that other people have written. `sklearn` has a lot of functions useful for machine learning, and we'll try out their function for nearest neighbors.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
train_inputs = train_data[:,:-1]
train_labels = train_data[:,-1]
neigh.fit(train_inputs, train_labels)

In [ ]:
test_inputs = test_data[:,:-1]
test_labels = test_data[:,-1]
neigh.score(test_inputs, test_labels)

**Do you get the same accuracies as you did for your own implementation?**

## 4. Decision Trees.

Now, we'll try using a decision tree classifier. We'll use sklearn again here.

### 4a: Building and training the tree.
First, we need to build the decision tree and fit it to our train data.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_inputs, train_labels)

### 4b. Testing the tree.
Now let's test our classifier on the test data.

In [ ]:
clf.score(test_inputs, test_labels)

### 4c. Viewing the tree.
If you're interested in seeing what the tree actually looks like, run the following cells. The tree will be saved in the file test.pdf.

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

In [ ]:
import os
os.environ["PATH"] += os.pathsep +'~/miniconda3/bin/graphviz/'

In [ ]:
dot_data = export_graphviz(clf, out_file=None, class_names=True, label='none', impurity=False) 
graph = graphviz.Source(dot_data) 
graph.render("test") 

### Extra Challenge.

"Extra challenge" sections are a more unguided exploration into the concepts we've discussed. You'll notice less scaffolding for the code -- try implementing these concepts from scratch, and feel free to ask your neighbors or an instructor if you have any questions!

Now that we've created some nearest neighbors classifiers, see how changing certain aspects of the classifier changes the classification accuracy. Some ideas of things to try:
- Change the train/test split..
- Try different values if $k$ -- what seems to give the best classification accuracy?
- Test both the euclidean and manhattan distance functions -- does one give a better classification accuracy than the other?